# FAA Aircraft Registration Database

In [1]:
import pandas as pd
import requests
import matplotlib as mpl
import geopandas as gpd
import descartes
import geojson
import json
import numpy as np
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_colwidth', 100)

In [2]:
# https://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/

In [3]:
master = pd.read_csv('/Users/mhustiles/data/data/FAA_Aircraft_Database/MASTER.txt', low_memory=False)

In [4]:
reference = pd.read_csv('/Users/mhustiles/data/data/FAA_Aircraft_Database/ACFTREF.txt', low_memory=False)

In [5]:
master.columns = master.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [6]:
reference.columns = reference.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [7]:
reference.loc[(reference['type_acft'] == '4'),'type']='Fixed wing single engine'
reference.loc[(reference['type_acft'] == '5'),'type']='Fixed wing multi engine'
reference.loc[(reference['type_acft'] == '6'),'type']='Rotorcraft'

In [8]:
reference.head()

,code,mfr,model,type_acft,type_eng,ac_cat,build_cert_ind,no_eng,no_seats,ac_weight,speed,unnamed:_11,type
0,0020901,AAR AIRLIFT GROUP INC,UH-60A,6,3,1,0,2,15,CLASS 3,0,nan,Rotorcraft
1,0030109,EXLINE ACE-C,ACE-C,4,1,1,1,1,1,CLASS 1,82,nan,Fixed wing single engine
2,003010D,DELEBAUGH,P,4,1,1,1,1,1,CLASS 1,82,nan,Fixed wing single engine
3,003010H,DAL PORTO,BABY ACE D,4,1,1,1,1,1,CLASS 1,82,nan,Fixed wing single engine
4,003010P,DUNN,BABY ACE,4,1,1,1,1,1,CLASS 1,82,nan,Fixed wing single engine


In [9]:
master.name = master.name.str.strip()
master.certification = master.certification.str.strip()
master.type_registrant = master.type_registrant.str.strip()

In [10]:
owners = master.groupby(['name', 'type_registrant', 'certification']).agg('size').reset_index(name='planes')

In [11]:
owners.head()

,name,type_registrant,certification,planes
0,,,,160
1,,,1,33
2,,,1B,2
3,,,1N,37
4,,,1NA,2


In [12]:
corptrans = owners[(owners['planes'] > 20) & (owners['certification'] == '1T')]

In [13]:
airlines = ['DELTA AIR LINES INC', 'AMERICAN AIRLINES INC', 'SKYWEST AIRLINES INC', 'SWIFT AIR LLC',\
'UNITED AIRLINES INC', 'SOUTHWEST AIRLINES CO', 'JETBLUE AIRWAYS CORP', 'HORIZON AIR INDUSTRIES',\
'ALASKA AIRLINES INC', 'REPUBLIC AIRWAYS INC', 'SPIRIT AIRLINES INC', 'MESA AIRLINES INC']

In [14]:
airlines

['DELTA AIR LINES INC',
 'AMERICAN AIRLINES INC',
 'SKYWEST AIRLINES INC',
 'SWIFT AIR LLC',
 'UNITED AIRLINES INC',
 'SOUTHWEST AIRLINES CO',
 'JETBLUE AIRWAYS CORP',
 'HORIZON AIR INDUSTRIES',
 'ALASKA AIRLINES INC',
 'REPUBLIC AIRWAYS INC',
 'SPIRIT AIRLINES INC',
 'MESA AIRLINES INC']

In [15]:
cargo = ['UNITED PARCEL SERVICE CO','FEDERAL EXPRESS CORP', 'FEDERAL EXPRESS CORPORATION',\
            'CARGO AIRCRAFT MANAGEMENT INC', 'ATLAS AIR INC']

In [16]:
cargo

['UNITED PARCEL SERVICE CO',
 'FEDERAL EXPRESS CORP',
 'FEDERAL EXPRESS CORPORATION',
 'CARGO AIRCRAFT MANAGEMENT INC',
 'ATLAS AIR INC']

In [17]:
private = ['NETJETS SALES INC', 'FLEXJET LLC']

In [18]:
corptrans.sort_values(by='planes', ascending=False).head()

,name,type_registrant,certification,planes
220089,WELLS FARGO TRUST CO NA TRUSTEE,3,1T,1231
54055,DELTA AIR LINES INC,3,1T,979
8943,AMERICAN AIRLINES INC,3,1T,910
211582,UNITED AIRLINES INC,3,1T,861
16036,BANK OF UTAH TRUSTEE,3,1T,787


In [19]:
airline_planes = master

In [20]:
airline_planes_slim = airline_planes[['name','n_number']]

In [21]:
airline_planes_slim.to_csv('output/airline_planes.csv', index=False)

### Airline planes

In [22]:
american = master[master['name'].str.contains('AMERICAN AIRLINES')]

In [23]:
american.iloc[583]

n_number                                                         766AN
serial_number                           32880                         
mfr_mdl_code                                                   1385213
eng_mfr_mdl                                                      54555
year_mfr                                                          2003
type_registrant                                                      3
name                                             AMERICAN AIRLINES INC
street                               1 SKYVIEW DR                     
street2                              MD 8B369                         
city                                                FORT WORTH        
state                                                               TX
zip_code                                                    76155     
region                                                               2
county                                                             439
countr

In [24]:
master['n_number'] = 'N' + master['n_number']

In [25]:
src = master.merge(reference, left_on='mfr_mdl_code', right_on='code')

In [26]:
src[src['n_number'] == 'N302FD']

,n_number,serial_number,mfr_mdl_code,eng_mfr_mdl,year_mfr,type_registrant,name,street,street2,city,...,type_acft,type_eng,ac_cat,build_cert_ind,no_eng,no_seats,ac_weight,speed,unnamed:_11,type
207667,N302FD,41239,0260106,52174,2011,5,LOS ANGELES CITY,C/O LOS ANGELES FIRE DEPARTMENT,AIR OPS SECTION 16617 ARMINTA ST,VAN NUYS,...,6,3,1,0,2,17,CLASS 3,0,nan,Rotorcraft


In [51]:
lapd = ['AIR SUPPORT DIVISION', 
        'LAPD', 
        'LOS ANGELES POLICE', 'LOS ANGELES POLICE DEPT AIR SUPPORT DIVISION']

In [52]:
lapd_county = ['LOS ANGELES COUNTY',
                'LOS ANGELES CITY',
                'LA COUNTY',
                'CONSOLIDATED FIRE PROTECTION', 
                'LACOUNTY', 
                'LAPD', 
                'LOS ANGELES POLICE',
                'LOS ANGELES POLICE DEPT AIR SUPPORT DIVISION',
                'COUNTY OF LOS ANGELES',
                'COUNTY OF LA',
                'CITY OF LOS ANGELES',
                'LOS ANGELES FIRE',
                'LOS ANGELES FIRE DEPARTMENT',
                'LOS ANGELES CITY']

In [53]:
la_cops_choppers = src[(src['name'].str.contains('|'.join(lapd_county))) \
                       & (src['state'] == 'CA') \
              & (src['type_aircraft'] == '6')]

In [54]:
la_safety_aircraft = src[(src['name'].str.contains('|'.join(lapd_county))) \
                       & (src['state'] == 'CA')]

In [55]:
la_safety_choppers_slim = la_cops_choppers\
    [[ 'n_number', 'name', 'mfr','model','street','city','year_mfr','no_seats' ]]

In [56]:
la_safety_aircraft_slim = la_safety_aircraft\
    [[ 'n_number', 'name', 'mfr','model','street','city','year_mfr','no_seats', 'type_aircraft' ]]

In [57]:
len(la_cops_choppers)

55

In [58]:
len(la_safety_aircraft_slim)

60

In [61]:
la_safety_aircraft_slim.sort_values('type_aircraft', ascending=True)

,n_number,name,mfr,model,street,city,year_mfr,no_seats,type_aircraft
168067,N6330C,LOS ANGELES COUNTY SHERIFFS DEPARTMENT,CESSNA,210N,3235 N LAKEWOOD BLVD,LONG BEACH,1980,6,4
185543,N732WQ,LOS ANGELES COUNTY SHERIFFS DEPARTMENT,CESSNA,T210M,3235 N LAKEWOOD BLVD,LONG BEACH,1977,6,4
129010,N52006,LOS ANGELES COUNTY OFFICE PUBLIC ADMINISTRATOR,NORTH AMERICAN,SNJ-5,320 W TEMPLE ST FL 8,LOS ANGELES,,2,4
242887,N851EM,LOS ANGELES POLICE DEPARTMENT,BEECH,C-12C,555 RAMIREZ ST STE 475,LOS ANGELES,1976,15,5
156916,N950MB,LOS ANGELES COUNTY SHERIFFS DEPARTMENT,BEECH,B200,AERO BUREAU,LONG BEACH,1990,11,5
183119,N303FD,LOS ANGELES CITY,AGUSTA SPA,AW139,C/0 LOS ANGELES FIRE DEPARTMENT,VAN NUYS,2008,17,6
183120,N305FD,CITY OF LOS ANGELES,AGUSTA SPA,AW139,C/O AIR OPERATIONS SECTION,VAN NUYS,2008,17,6
206476,N950LA,COUNTY OF LOS ANGELES,AMERICAN EUROCOPTER CORP,AS350B2,AERO BUREAU,LONG BEACH,2010,7,6
206477,N951LA,COUNTY OF LOS ANGELES,AMERICAN EUROCOPTER CORP,AS350B2,C/O SHERIFF'S DEPARTMENT,LONG BEACH,2010,7,6
206478,N953LA,COUNTY OF LOS ANGELES,AMERICAN EUROCOPTER CORP,AS350B2,C/O SHERIFFS DEPT,LONG BEACH,2010,7,6


In [32]:
la_cops_choppers.to_csv('output/la_safety_choppers_slim.csv', index=False)

In [33]:
la_cops_choppers.n_number.unique().tolist()

['N306FD',
 'N664PD',
 'N3202Q',
 'N601CC',
 'N120LA',
 'N73985',
 'N213PF',
 'N221LA',
 'N223LA',
 'N267LA',
 'N472LA',
 'N703WP',
 'N704WP',
 'N110LA',
 'N14LA ',
 'N21844',
 'N228LA',
 'N229LA',
 'N232LA',
 'N233LA',
 'N661PD',
 'N662PD',
 'N665PD',
 'N913WB',
 'N961SD',
 'N963SD',
 'N966SD',
 'N301FD',
 'N304FD',
 'N303FD',
 'N305FD',
 'N950LA',
 'N951LA',
 'N953LA',
 'N954LA',
 'N955LA',
 'N956LA',
 'N957LA',
 'N958LA',
 'N959LA',
 'N960LA',
 'N961LA',
 'N962LA',
 'N302FD',
 'N668PD',
 'N17LA ',
 'N18LA ',
 'N15LA ',
 'N160LA',
 'N190LA',
 'N821LA',
 'N822LA',
 'N950SG',
 'N951LB',
 'N952JH']

---

### Bobcat fire aircraft

In [34]:
bobcat_src = pd.read_csv('input/aircraft.csv')

In [35]:
bobcat_src.n_number = bobcat_src.n_number.str.strip()

In [36]:
bobcat = bobcat_src.merge(master, on='n_number')

In [37]:
bobcat_merge = bobcat.merge(reference, left_on='mfr_mdl_code', right_on='code')

In [38]:
bobcat_slim = bobcat_merge[[ 'n_number', 'name', 'street', 'city', 'type', 'mfr', 'model', 'no_seats']]

In [39]:
bobcat_slim.head()

,n_number,name,street,city,type,mfr,model,no_seats
0,N409DF,USDA FOREST SERVICE FEPP,5500 PRICE AVE,MCCLELLAN,Fixed wing multi engine,NORTH AMERICAN,OV-10A,2
1,N415DF,USDA FOREST SERVICE FEPP,5500 PRICE AVE,MCCLELLAN,Fixed wing multi engine,NORTH AMERICAN,OV-10A,2
2,N463DF,STATE OF CALIFORNIA - CAL FIRE,5500 PRICE AVE,MCCLELLAN,Fixed wing multi engine,RAYTHEON AIRCRAFT COMPANY,A200CT,15
3,N110LA,CONSOLIDATED FIRE PROTECTION,12605 OSBORNE ST,PACOIMA,Rotorcraft,BELL,412EP,15
4,N436DF,USDA FOREST SERVICE - FEPP,5500 PRICE AVE,MCCLELLAN,Fixed wing multi engine,MARSH AVIATION,S-2F3AT,5


In [40]:
bobcat_slim.to_csv('output/aircraft.csv', index=False)